<a href="https://colab.research.google.com/github/juliana-zh/ML/blob/main/NLP_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задача машинного обучения. Обработка ествестенного языка (NLP)

Цель: обучить модель для задачи обработки естественного языка, а именно классификации новостных текстов по темам.

Дано: набор данных с 18000 новостных текстов, сгруппированных по 20 темам.

In [ ]:
# libraries
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Подгружаем новостные тексты и формируем из них тренировочную выборку:

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

Смотрим, какие есть темы:

In [ ]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Сколько у нас всего новостных текстов:

In [ ]:
newsgroups_train.filenames.shape

(11314,)

In [ ]:
newsgroups_train.filenames[0]

'/root/scikit_learn_data/20news_home/20news-bydate-train/rec.autos/102994'

#### Рассмотрим подвыборку

Возьмем не все темы, а подвыборку их (четыре темы). Прогрузим новостные тексты по этим темам. Посмотрим, сколько всего таких текстов:

In [ ]:
categories = ['alt.atheism', 'talk.religion.misc',
              'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      categories=categories)
newsgroups_train.filenames.shape

(2034,)

Пример новостного текста:

In [ ]:
print(newsgroups_train.data[0])

From: rych@festival.ed.ac.uk (R Hawkes)
Subject: 3DS: Where did all the texture rules go?
Lines: 21

Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

Rycharde Hawkes				email: rych@festival.ed.ac.uk
Virtual Environment Laboratory
Dept. of Psychology			Tel  : +44 31 650 3426
Univ. of Edinburgh			Fax  : +44 31 667 0150



Метки тем (четыре штуки: от 0 до 3)

In [ ]:
newsgroups_train.target[:10]

array([1, 3, 2, 0, 2, 0, 2, 1, 2, 1])

#### TF-IDF

TF-IDF - это мера, показывающая специфичность данного слова для данного конкретного текста.

$n_{\mathbb{d}\mathbb{w}}$ - число вхождений слова $\mathbb{w}$ в документ $\mathbb{d}$;<br>
$N_{\mathbb{w}}$ - число документов, содержащих $\mathbb{w}$;<br>
$N$ - число документов; <br><br>

$p(\mathbb{w}, \mathbb{d}) = N_{\mathbb{w}} / N$ - вероятность наличия слова $\mathbb{w}$ в любом документе $\mathbb{d}$
<br>
$P(\mathbb{w}, \mathbb{d}, n_{\mathbb{d}\mathbb{w}}) = (N_{\mathbb{w}} / N)^{n_{\mathbb{d}\mathbb{w}}}$ - вероятность встретить $n_{\mathbb{d}\mathbb{w}}$ раз слово $\mathbb{w}$ в документе $\mathbb{d}$<br><br>

$-\log{P(\mathbb{w}, \mathbb{d}, n_{\mathbb{d}\mathbb{w}})} = n_{\mathbb{d}\mathbb{w}} \cdot \log{(N / N_{\mathbb{w}})} = TF(\mathbb{w}, \mathbb{d}) \cdot IDF(\mathbb{w})$<br><br>

$TF(\mathbb{w}, \mathbb{d}) = n_{\mathbb{d}\mathbb{w}}$ - term frequency;<br>
$IDF(\mathbb{w}) = \log{(N /N_{\mathbb{w}})}$ - inverted document frequency;

#### Векторизуем эти тексты с помощью меры TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Каждому слову поставим в соответствие вектор:

In [ ]:
vectorizer = TfidfVectorizer(lowercase=False)
vectors = vectorizer.fit_transform(newsgroups_train.data)

In [ ]:
vectors.shape

(2034, 42307)

Видим, что у нас на 2034 документа приходится 42307 векторов (это представления слов).

И каждое слово - это признак.

In [ ]:
vectorizer.get_feature_names_out()[:10]

array(['00', '000', '0000', '00000', '000000', '000005102000', '000021',
       '000062David42', '0000VEC', '0001'], dtype=object)

Видим, что слова в нашем наборе редкие, малоиспользуемые, неинформативные: '00', '000', '0000', '00000', '000000', '000005102000', '000021', '000062David42', '0000VEC', '0001'

Нам лучше обучать модель на более информативных словах. Для этого выставим пороги - параметры max_df и min_df.

max_df = 0.9 - значит, игнорируем слова, которые встречаются в более чем 90% случаев
min_df = 0.03 - значит, игнорируем слова, которые встречаются в менее чем 3% случаев

Обучим модель:

In [ ]:
# ngram_range
vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=0.03, max_df=0.9)
vectors = vectorizer.fit_transform(newsgroups_train.data)


In [ ]:
vectors.shape

(2034, 1236)

Видим, что признаков (слов) у нас осталось значительно меньше.

Создаем фукцию preproc_nltk, которая убирает стоп-слова (неинформативные, чатсо употребимые слова, например: "the") и производит лемматизацию (приведение слов к леммам - начальным формам - таким, как в словаре)

In [ ]:
# стоп-слова, preproc
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
nltk.download('wordnet')
wnl = nltk.WordNetLemmatizer()

def preproc_nltk(text):
    #text = re.sub(f'[{string.punctuation}]', ' ', text)
    return ' '.join([wnl.lemmatize(word) for word in word_tokenize(text.lower()) if word not in stopWords])

st = "Oh, I think I ve landed Where there are miracles at work,  For the thirst and for the hunger Come the conference of birds"
preproc_nltk(st)

[nltk_data] Downloading package wordnet to /root/nltk_data...


'oh , think landed miracle work , thirst hunger come conference bird'

Замеряем время выполнения векторизации всей тренировочной выборки с использованием предобработки:

In [ ]:
%%time
vectorizer = TfidfVectorizer(preprocessor=preproc_nltk)
vectors = vectorizer.fit_transform(newsgroups_train.data)

CPU times: user 9.95 s, sys: 8.47 ms, total: 9.95 s
Wall time: 10.3 s


А теперь воспользуемся еще одним способом предобработки данных - с использованием библиотеки spacy. Ей работа основана на использовании нейросетей.

In [ ]:
# preproc_spacy
import spacy
nlp = spacy.load("en_core_web_sm")
texts = newsgroups_train.data.copy()

def preproc_spacy(text):
    spacy_results = nlp(text)
    return ' '.join([token.lemma_ for token in spacy_results if token.lemma_ not in stopWords])
preproc_spacy(st)

'oh , I think I land miracle work ,   thirst hunger come conference bird'

Здесь мы замеряем время предобработки всей тренировочной выборки и векторизации ее:

In [ ]:
%%time
new_texts = []
for doc in nlp.pipe(texts, batch_size=32, n_process=3, disable=["parser", "ner"]):
    new_texts.append(' '.join([tok.lemma_ for tok in doc if tok.lemma not in stopWords]))
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(new_texts)

CPU times: user 16 s, sys: 346 ms, total: 16.3 s
Wall time: 1min 33s


Итоговая модель:

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_df=0.5, max_features=1000)
vectors = vectorizer.fit_transform(new_texts)

Здесь мы берем текст, предобработанный spacy, векторизация осуществляется с помощью TfidfVectorizer.

Посмотрим на признаки (слова):

In [ ]:
vectorizer.get_feature_names_out()[::100]

array(['000', 'au', 'christ', 'engineering', 'human', 'look', 'of this',
       'report', 'tell', 'universe'], dtype=object)

#### Обучим модель на полученных признаках:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.linear_model import SGDClassifier

dense_vectors = vectors.todense()
X_train, X_test, y_train, y_test= train_test_split(dense_vectors, newsgroups_train.target, test_size=0.2, random_state=0)
y_train.shape, y_test.shape

((1627,), (407,))

Проверяем тип данных:

In [ ]:
type(X_train)

numpy.matrix

Преобразуем тип данных:

In [ ]:
import numpy as np

X_train = np.squeeze(np.asarray(X_train))
y_train = np.squeeze(np.asarray(y_train))

X_test = np.squeeze(np.asarray(X_test))
y_test = np.squeeze(np.asarray(y_test))

Обучаем модель классификатора Support Vector Classification. Замеряем время:

In [ ]:
%%time
svc = svm.SVC()
svc.fit(X_train, y_train)

CPU times: user 1.41 s, sys: 5.13 ms, total: 1.42 s
Wall time: 1.42 s


SVC()

Метрика точности для классификатора Support Vector Classification:

In [ ]:
accuracy_score(y_test, svc.predict(X_test))

0.9262899262899262

Обучаем модель линейного классификатора с использованием стохатического градиентного спуска (stochastic gradient descent (SGD)). Считаем точность на тестовой выборке:

In [ ]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
accuracy_score(y_test, sgd.predict(X_test))

0.918918918918919

С благодарностью, Юля.